In [1]:
import numpy as np
import pandas as pd

import conv_model as conv
import batch_hist as hist
import tensorflow as tf

In [2]:
#### User input ####

## path of the train and test data
train_data = "../data/digit-recognizer/train.csv"
test_data = "../data/digit-recognizer/test.csv"

In [3]:
## read data into Dataframe
df_train = pd.read_csv(train_data)
df_train.info()

df_test = pd.read_csv(test_data)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 167.5 MB


In [4]:
## get the first 400 rows of the data
#df_train = df_train.iloc[:400]
#print(len(df_train.columns))

In [5]:
## function to create input images
def create_input_images(df, start = 1):
    im_arr = []
    for row in range(0, len(df.index)):
        ## get the image values
        im_flat = np.array(df.iloc[row, start:])
        
        ## create the image frame
        image = np.reshape(im_flat, (28, 28, 1))
    
        im_arr.append(image)

    return np.array(im_arr, dtype = np.float32)

In [6]:
## create the arrays of input images
im_arr = create_input_images(df_train)
im_arr_test = create_input_images(df_test, start = 0)

In [7]:
from tensorflow.keras.losses import CategoricalCrossentropy

#### User input ####
learn_rate = 3.0e-3

In [8]:
## create the model and instantiate the batch callbacks
model1 = conv.ConvModel(conv_layer = 'Conv2D', drop_rate = 0.3)
batch_history = hist.BatchHistory()

In [10]:
## compile the model
Adam_opt = tf.keras.optimizers.Adam(learning_rate = learn_rate)
model1.compile(optimizer = Adam_opt, loss = CategoricalCrossentropy(), metrics = ['accuracy'])

In [11]:
## create training input and one_hot encode
Y = df_train["label"].to_numpy()
Y_oh = tf.one_hot(Y, 10)
im_arr.shape

(42000, 28, 28, 1)

In [12]:
## fit the model (currently just testing the design, no final run yet)
model1.fit(im_arr, Y_oh, batch_size = 64, epochs = 3, callbacks = [batch_history])

Epoch 1/10


2024-04-12 08:29:45.600533: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


657/657 [==============================] - 15s 22ms/step - loss: 0.3436 - accuracy: 0.8900
Epoch 2/10
657/657 [==============================] - 15s 23ms/step - loss: 0.1294 - accuracy: 0.9610
Epoch 3/10
657/657 [==============================] - 16s 24ms/step - loss: 0.1140 - accuracy: 0.9654
Epoch 4/10
657/657 [==============================] - 17s 26ms/step - loss: 0.1005 - accuracy: 0.9687
Epoch 5/10
657/657 [==============================] - 18s 28ms/step - loss: 0.1015 - accuracy: 0.9694
Epoch 6/10
657/657 [==============================] - 19s 28ms/step - loss: 0.0913 - accuracy: 0.9715
Epoch 7/10
657/657 [==============================] - 19s 29ms/step - loss: 0.0935 - accuracy: 0.9702
Epoch 8/10
657/657 [==============================] - 20s 30ms/step - loss: 0.0880 - accuracy: 0.9728
Epoch 9/10
657/657 [==============================] - 20s 30ms/step - loss: 0.0870 - accuracy: 0.9720
Epoch 10/10
657/657 [==============================] - 20s 30ms/step - loss: 0.0819 - accurac

In [16]:
## plot the accuracy evolution during training
import matplotlib.pyplot as plt
plt.plot(batch_history.accuracies)
plt.title('model accuracy')

plt.xscale('log')
plt.yscale('log')

plt.ylabel('accuracy')
plt.xlabel('batch')

plt.show()

In [ ]:
## print the model summary
model1.summary()

In [ ]:
## predict for the test data
preds = model1.predict(im_arr_test)

In [ ]:
## add test predictions to the test dataframe
#print(preds.shape)
#print(len(np.argmax(preds, axis = 1)))
#print(len(np.arange(1, len(df_test.index)+1)))
#print(df_test.columns)

df_test["Label"] = np.argmax(preds, axis = 1)
df_test["ImageId"] = np.arange(1, len(df_test.index)+1)

In [ ]:
## path of the test predictions
test_pred_path = "../data/digit-recognizer/test_preds4.csv"

## write results to csv file
#to_write = ["ImageId", "Label"]
#df_test.to_csv(test_pred_path, columns = to_write, index = False)